In [72]:
from openai import OpenAI
from google_images_search import GoogleImagesSearch
import json


In [62]:
try:
    with open('openai_key.txt', 'r') as file:
        openai_key = file.read()
except Exception as e:
        print(e)


try:
    with open('google_api_key.txt', 'r') as file:
        google_api_key = file.read()
except Exception as e:
        print(e)

try:
    with open('google_cse.txt', 'r') as file:
        google_cse = file.read()
except Exception as e:
        print(e)


In [63]:
client = OpenAI(
        api_key=openai_key,
    )

In [76]:
def get_images(query):
    gis = GoogleImagesSearch(google_api_key, google_cse)
    _search_params = {
        'q': query,
        'num': 1,
        'fileType': 'jpg|gif|png',
        'imgType': 'photo',
        'searchType': 'image',
    }
    gis.search(search_params=_search_params)
    for image in gis.results():
        return image.url

In [77]:
def generate_response(prompt,location="Lausanne"):   

    completion = client.chat.completions.create(
    messages=[
    {"role": "system", "content": f"You are travel advisor in Switzerland. The client will describe their preferences and you will suggest a place to visit. you should output a json containing a list results of up to 5 results containing:\
    name: the name of the place,\n \
    address: the adress\n \
    description: a short description of the place\n \
     image: keep this empty for now. \n \
     You should prioritize places that are close to {location}."},
    {"role": "user", "content": prompt},
  ],
    model="gpt-3.5-turbo",
)
    query_results = json.loads(completion.choices[0].message.content)
    # add an empty image field to the results
    
    for result in query_results['results']:
        image = get_images(result['name'])
        result['image'] = image
        
    
    
    return query_results

{'results': [{'name': 'Uetliberg', 'address': 'Uetliberg, 8143 Üetliberg, Switzerland', 'description': 'Uetliberg is a mountain in the Swiss plateau, part of the Albis chain, rising to 869 meters. It offers stunning panoramic views of Zurich and the surrounding region.', 'image': 'https://www.zuerich.com/sites/default/files/web_zuerich_uetliberg_1280x960_9573.jpg'}, {'name': 'Rigi Mountain', 'address': 'Rigi, 6410 Rigi Kaltbad, Switzerland', 'description': 'Rigi Mountain is known as the queen of the mountains and offers breathtaking views of the Swiss Alps and neighbouring lakes. It is easily accessible from Zurich and offers numerous hiking trails for all levels of difficulty.', 'image': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/b7/62/f9/mt-rigi.jpg?w=1200&h=1200&s=1'}, {'name': 'Glarus Thrust', 'address': 'Glarus Thrust, 8762 Selun, Switzerland', 'description': 'Glarus Thrust is a geologic phenomenon that can be explored through various hiking trails. It showcases t